In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from keras.utils import np_utils
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
import pandas as pd
import csv
import numpy as np
import sys
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

img_rows, img_cols = 28, 28
img_channels = 1
num_pixels = img_cols*img_rows
num_classes = 10
num_trainImages = 42000
num_testImages = 28000

yTrain = np.ones((num_trainImages))
xTrain = np.ones((num_trainImages,img_channels,img_cols,img_rows))
counter = 0
skip = True

# train.csv should be in the same folder as this file
trainFile = open('datasets/train.csv')
csv_file = csv.reader(trainFile)
for row in csv_file:
    if (skip == True):
        skip = False
        continue
    yTrain[counter] = row[0]
    temp = np.ones((1,num_pixels))
    for num in range(1,num_pixels):
        temp[0,num - 1] = row[num]
    temp = (temp - np.mean(temp))/(np.max(temp) - np.min(temp))
    temp = np.reshape(temp, (img_rows,img_cols))
    xTrain[counter,0,:,:] = temp
    counter += 1

yTest = np.ones((num_testImages))
xTest = np.ones((num_testImages,img_channels,img_cols,img_rows))
skip2 = True
counter2 = 0

testFile = open('datasets/test.csv')
csv_file2 = csv.reader(testFile)
for row in csv_file2:
    if (skip2 == True):
        skip2 = False
        continue
    yTest[counter2] = row[0]
    temp = np.ones((1,num_pixels))
    for num in range(1,num_pixels):
        temp[0,num - 1] = row[num]
    temp = (temp - np.mean(temp))/(np.max(temp) - np.min(temp))
    temp = np.reshape(temp, (img_rows,img_cols))
    xTest[counter2,0,:,:] = temp
    counter2 += 1

# Convert class vectors to binary class matrices
yTrain = np_utils.to_categorical(yTrain, num_classes)
yTest = np_utils.to_categorical(yTest, num_classes)

In [7]:
# NETWORK ARCHITECTURE
input_img = Input(shape= (img_channels,img_rows, img_cols))
x = Conv2D(32, kernel_size = (3,3), padding = 'Same', activation='relu')(input_img)
x = Conv2D(64,  kernel_size = (3,3), padding = 'Same', activation='relu')(x)
x = Conv2D(64,  kernel_size = (3,3), padding = 'Same', activation='relu')(x)
x = MaxPooling2D(pool_size=(1,1))(x)
x = Flatten()(x)
predictions = Dense(64, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(predictions)

model = Model(input_img, predictions)

In [8]:
# SPLIT THE DATASET
xTrain.shape
vTrain = xTrain[33600:42000,:,:,:]
vTest = yTrain[33600:42000,:]
xTrain = xTrain[:33600,:,:,:]
yTrain = yTrain[:33600,:]

In [15]:
# TRAINING
model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])
model.fit(xTrain, yTrain, batch_size=32, nb_epoch=8,validation_data=(vTrain, vTest),shuffle=True)

W0316 20:34:53.519562 140088041698688 training.py:686] The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 33600 samples, validate on 28000 samples
Epoch 1/8
33600/33600 [==============================] - 13s 376us/sample - loss: 2.2968 - accuracy: 0.1265 - val_loss: 2.2822 - val_accuracy: 0.4253
Epoch 2/8
33600/33600 [==============================] - 9s 279us/sample - loss: 2.2883 - accuracy: 0.2034 - val_loss: 2.2760 - val_accuracy: 0.3932
Epoch 3/8
33600/33600 [==============================] - 9s 267us/sample - loss: 2.2764 - accuracy: 0.2753 - val_loss: 2.2644 - val_accuracy: 0.3746
Epoch 4/8
33600/33600 [==============================] - 9s 266us/sample - loss: 2.2586 - accuracy: 0.3258 - val_loss: 2.2453 - val_accuracy: 0.3602
Epoch 5/8
33600/33600 [==============================] - 10s 290us/sample - loss: 2.2310 - accuracy: 0.3624 - val_loss: 2.2182 - val_accuracy: 0.3339
Epoch 6/8
33600/33600 [==============================] - 9s 281us/sample - loss: 2.1880 - accuracy: 0.3999 - val_loss: 2.1799 - val_accuracy: 0.3082
Epoch 7/8
33600/33600 [==============================]

In [16]:
results = np.zeros((num_testImages,2))
for num in range(1,num_testImages + 1):	
    results[num - 1,0] = num

In [18]:
y_prob = model.predict(xTest) 
temp = y_prob.argmax(axis=-1)

In [19]:
for num in range(0,num_testImages):	
    results[num,1] = temp[num]
# Results saved in this text file
np.savetxt('submission2.csv', results, delimiter=',', fmt = '%i')  
results = pd.np.array(results)